In [88]:

import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix, log_loss
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

In [89]:
# Set random seed for reproducibility
np.random.seed(42)

In [90]:
# --- Model Training and Evaluation ---
# Load dataset
df = pd.read_csv("face_mimic_df.csv")

In [91]:
# Define features
features = ['AU_01_t12', 'AU_06_t12', 'AU_12_t12', 'AU_04_t13', 'AU_07_t13', 
            'AU_09_t13', 'AU_01_t14', 'AU_02_t14', 'AU_04_t14', 'age', 'gender']
X = df[features].dropna()
y = df.loc[X.index, 'diagnosed']

In [92]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [93]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [94]:
# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

In [95]:
# Perform stratified 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'accuracy': 'accuracy',
    'f1': 'f1',
    'precision': 'precision',
    'recall': 'recall',
    'roc_auc': 'roc_auc'}

In [96]:
# Custom function to compute log loss and accuracies
train_accuracies, val_accuracies = [], []
train_losses, val_losses = [], []
y_true_val, y_pred_val, y_pred_proba_val = [], [], []

for train_idx, val_idx in cv.split(X_resampled, y_resampled):
    X_train, X_val = X_resampled[train_idx], X_resampled[val_idx]
    y_train, y_val = y_resampled[train_idx], y_resampled[val_idx]
    
    # Train model
    rf_model.fit(X_train, y_train)
    
    # Predict on training and validation sets
    y_train_pred = rf_model.predict(X_train)
    y_val_pred = rf_model.predict(X_val)
    y_train_proba = rf_model.predict_proba(X_train)
    y_val_proba = rf_model.predict_proba(X_val)
    
    # Compute accuracy
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    val_accuracies.append(accuracy_score(y_val, y_val_pred))
    
    # Compute log loss (proxy for loss)
    train_losses.append(log_loss(y_train, y_train_proba))
    val_losses.append(log_loss(y_val, y_val_proba))
    
    # Store validation predictions for AUPRC and confusion matrix
    y_true_val.extend(y_val)
    y_pred_val.extend(y_val_pred)
    y_pred_proba_val.extend(y_val_proba[:, 1])

In [97]:
# Compute AUPRC for validation data
precision, recall, _ = precision_recall_curve(y_true_val, y_pred_proba_val)
auprc = auc(recall, precision)

In [35]:
# Print cross-validation results
print("Cross-Validation Results (SMOTE-balanced data):")
print(f"Mean Training Accuracy: {np.mean(train_accuracies):.4f} (±{np.std(train_accuracies):.4f})")
print(f"Mean Validation Accuracy: {np.mean(val_accuracies):.4f} (±{np.std(val_accuracies):.4f})")
print(f"Mean Training Log Loss: {np.mean(train_losses):.4f} (±{np.std(train_losses):.4f})")
print(f"Mean Validation Log Loss: {np.mean(val_losses):.4f} (±{np.std(val_losses):.4f})")
print(f"Mean F1 Score: {np.mean(cross_validate(rf_model, X_resampled, y_resampled, cv=cv, scoring='f1')['test_score']):.4f}")
print(f"Mean Precision: {np.mean(cross_validate(rf_model, X_resampled, y_resampled, cv=cv, scoring='precision')['test_score']):.4f}")
print(f"Mean Recall: {np.mean(cross_validate(rf_model, X_resampled, y_resampled, cv=cv, scoring='recall')['test_score']):.4f}")
print(f"Mean ROC AUC: {np.mean(cross_validate(rf_model, X_resampled, y_resampled, cv=cv, scoring='roc_auc')['test_score']):.4f}")
print(f"AUPRC: {auprc:.4f}")

Cross-Validation Results (SMOTE-balanced data):
Mean Training Accuracy: 1.0000 (±0.0000)
Mean Validation Accuracy: 0.9271 (±0.0183)
Mean Training Log Loss: 0.0745 (±0.0016)
Mean Validation Log Loss: 0.2380 (±0.0144)
Mean F1 Score: 0.9294
Mean Precision: 0.9049
Mean Recall: 0.9563
Mean ROC AUC: 0.9850
AUPRC: 0.9845


In [98]:
# Evaluate on original (imbalanced) dataset
rf_model.fit(X_resampled, y_resampled)
y_pred_original = rf_model.predict(X_scaled)
y_pred_proba_original = rf_model.predict_proba(X_scaled)[:, 1]
precision_orig, recall_orig, _ = precision_recall_curve(y, y_pred_proba_original)
auprc_orig = auc(recall_orig, precision_orig)

In [37]:
print("\nPerformance on Original (Imbalanced) Data:")
print(f"Accuracy: {accuracy_score(y, y_pred_original):.4f}")
print(f"F1 Score: {f1_score(y, y_pred_original):.4f}")
print(f"Precision: {precision_score(y, y_pred_original):.4f}")
print(f"Recall: {recall_score(y, y_pred_original):.4f}")
print(f"ROC AUC: {roc_auc_score(y, y_pred_proba_original):.4f}")
print(f"AUPRC: {auprc_orig:.4f}")


Performance on Original (Imbalanced) Data:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
ROC AUC: 1.0000
AUPRC: 1.0000


In [15]:
# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 6))
plt.plot(range(1, 6), train_accuracies, label='Training Accuracy', marker='o')
plt.plot(range(1, 6), val_accuracies, label='Validation Accuracy', marker='o')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Across Folds')
plt.legend()
plt.grid(True)
plt.savefig('accuracy_plot.png', dpi=300)
plt.close()

In [16]:
# Plot Training and Validation Loss
plt.figure(figsize=(8, 6))
plt.plot(range(1, 6), train_losses, label='Training Log Loss', marker='o')
plt.plot(range(1, 6), val_losses, label='Validation Log Loss', marker='o')
plt.xlabel('Fold')
plt.ylabel('Log Loss')
plt.title('Training and Validation Log Loss Across Folds')
plt.legend()
plt.grid(True)
plt.savefig('loss_plot.png', dpi=300)
plt.close()

In [16]:
# Plot Confusion Matrix (Original Data)
cm = confusion_matrix(y, y_pred_original)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Healthy', 'PD'], yticklabels=['Healthy', 'PD'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Original Imbalanced Data)')
plt.savefig('confusion_matrix.png', dpi=300)
plt.close()

In [99]:
# Save model and scaler for real-time use
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [18]:
import mediapipe as mp
print(mp.__version__)

0.10.21


In [100]:
# --- Real-Time AU Extraction and Prediction ---
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, 
                                  min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [101]:
# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

In [102]:
# Placeholder for AU extraction (simplified, as MediaPipe doesn't directly provide FACS AUs)
def extract_au_features(landmarks):
    if not landmarks:
        return np.zeros(9)  # Return zeros if no face detected
    
    # Example landmark indices (MediaPipe Face Mesh 468-point model)
    left_eye = landmarks.landmark[159]  # Upper eyelid
    left_brow = landmarks.landmark[70]  # Inner eyebrow
    right_eye = landmarks.landmark[386]
    right_brow = landmarks.landmark[300]
    cheek_left = landmarks.landmark[205]
    lip_corner_left = landmarks.landmark[61]
    lip_corner_right = landmarks.landmark[291]
    outer_brow_left = landmarks.landmark[66]
    
    # AU_01: Inner brow raiser
    au_01 = np.mean([np.abs(left_brow.y - left_eye.y), np.abs(right_brow.y - right_eye.y)])
    # AU_06: Cheek raiser
    au_06 = np.abs(cheek_left.y - left_eye.y)
    # AU_12: Lip corner puller
    au_12 = np.mean([np.abs(lip_corner_left.x - lip_corner_right.x)])
    # AU_04: Brow lowerer
    au_04 = np.abs(left_brow.y - right_brow.y)
    # AU_07: Lid tightener
    au_07 = np.abs(left_eye.y - landmarks.landmark[145].y)
    # AU_09: Nose wrinkler (placeholder)
    au_09 = 0.0
    # AU_02: Outer brow raiser
    au_02 = np.abs(outer_brow_left.y - left_eye.y)
    
    return np.array([au_01, au_06, au_12, au_04, au_07, au_09, au_01, au_02, au_04])

In [103]:
# Real-time prediction
au_buffer = []  # Buffer to store AU features
max_buffer_size = 150  # ~5 seconds at 30 fps
user_age = 60  # Placeholder
user_gender = 1  # Placeholder (1=male, 0=female)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)

# Define custom connections for specified AUs
custom_connections = [
    # AU_01_t12, AU_01_t14 (Inner Brow Raiser)
    (70, 71), (71, 63), (63, 105), (105, 70),  # Left inner brow
    (300, 299), (299, 293), (293, 334), (334, 300),  # Right inner brow
    (70, 300),  # Connect left and right
    # AU_06_t12 (Cheek Raiser)
    (205, 147), (147, 187), (187, 205),  # Left cheek
    (425, 367), (367, 407), (407, 425),  # Right cheek
    (205, 425),  # Connect left and right
    # AU_12_t12 (Lip Corner Puller)
    (61, 291),  # Connect left and right lip corners
    # AU_04_t13, AU_04_t14 (Brow Lowerer)
    (66, 107), (107, 297), (297, 332), (332, 66),  # Left outer brow
    (55, 285), (285, 296), (296, 54), (54, 55),  # Right outer brow
    (66, 55),  # Connect left and right
    # AU_07_t13 (Lid Tightener)
    (159, 145), (145, 386), (386, 374), (374, 159),  # Left and right eyelids
    (159, 386),  # Connect left and right
    # AU_09_t13 (Nose Wrinkler)
    (6, 197), (197, 168), (168, 6),  # Nose bridge and sides
    # AU_02_t14 (Outer Brow Raiser)
    (66, 55), (55, 107), (107, 285), (285, 66),  # Outer brow connection
    (66, 285)  # Connect left and right
]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    
    # Create a copy of the frame for text rendering
    frame_with_text = frame.copy()
    
    # Convert frame to RGB for MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if face_mesh is not None:
        results = face_mesh.process(frame_rgb)
    else:
        print("Error: FaceMesh is not initialized.")
        break
    
    # Extract AU features
    au_features = np.zeros(9)
    if results and results.multi_face_landmarks:
        au_features = extract_au_features(results.multi_face_landmarks[0])
    
    # Append to buffer
    au_buffer.append(au_features)
    if len(au_buffer) > max_buffer_size:
        au_buffer.pop(0)
    
    # Compute average AU features and predict (even with partial buffer)
    pred = 0  # Default to Healthy if no prediction
    pred_proba = 0.0
    if au_buffer and len(au_buffer) >= 1:
        avg_au = np.mean(au_buffer, axis=0)
        input_features = np.append(avg_au, [user_age, user_gender])
        try:
            input_scaled = scaler.transform([input_features])
            pred = rf_model.predict(input_scaled)[0]
            pred_proba = rf_model.predict_proba(input_scaled)[0][1]
        except Exception as e:
            print(f"Prediction error: {e}")
    
    # Display result on the text frame
    label = "PD" if pred == 1 else "Healthy"
    color = (0, 0, 255) if pred == 1 else (0, 255, 0)
    cv2.putText(frame_with_text, f"Status: {label} ({pred_proba:.2f})", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # Draw custom connections for specified AU landmarks with error handling
    if results and results.multi_face_landmarks:
        try:
            frame_with_lines = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)  # Separate frame for drawing
            for connection in custom_connections:
                start_point = results.multi_face_landmarks[0].landmark[connection[0]]
                end_point = results.multi_face_landmarks[0].landmark[connection[1]]
                start_pos = (int(start_point.x * frame_with_lines.shape[1]), int(start_point.y * frame_with_lines.shape[0]))
                end_pos = (int(end_point.x * frame_with_lines.shape[1]), int(end_point.y * frame_with_lines.shape[0]))
                # Check if coordinates are valid
                if (0 <= start_pos[0] < frame_with_lines.shape[1] and 0 <= start_pos[1] < frame_with_lines.shape[0] and
                    0 <= end_pos[0] < frame_with_lines.shape[1] and 0 <= end_pos[1] < frame_with_lines.shape[0]):
                    cv2.line(frame_with_lines, start_pos, end_pos, (0, 255, 0), thickness=1)
                    cv2.circle(frame_with_lines, start_pos, 1, (0, 255, 0), -1)
                    cv2.circle(frame_with_lines, end_pos, 1, (0, 255, 0), -1)
        except Exception as e:
            print(f"Drawing error: {e}")
    
    # Combine the text frame with the lines frame
    if 'frame_with_lines' in locals():
        frame_with_text = cv2.addWeighted(frame_with_text, 1, frame_with_lines, 1, 0)
    
    # Show frame
    cv2.imshow('PD Detection', frame_with_text)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler wa

In [104]:
# Cleanup
cap.release()
cv2.destroyAllWindows()
face_mesh.close()

In [105]:
# Save AU buffer for debugging
np.save('au_buffer.npy', np.array(au_buffer))